# 1)SET UP :

In [ ]:
!pip install git+https://github.com/openai/whisper.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-878doaa3
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-878doaa3
  Resolved https://github.com/openai/whisper.git to commit 248b6cb124225dd263bb9bd32d060b6517e067f8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.0 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=798092 sha256=f2fbd4538981b0feb8a0d6131b9f35de6d3b8b4c3fbe60d44fb808587043c9de
  Stored in directory: /tmp/pip-ephem-wheel-cache-0owoijo8/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


In [ ]:
from google.colab import drive

drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
model_name = "openai/whisper-small"

#2)Preprocessing ;

```
# This is formatted as code
```



### We are going to use google drive where we upoaded our dataset

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 12.1 MB/s eta 0:00:00


In [ ]:
 !ls {directory}

In [ ]:
import os
import re
import pandas as pd
import librosa
# define the directory where our files are located
directory = "/content/gdrive/MyDrive/Colab Notebooks/darija/"
data = pd.read_csv(directory + 'texts/train.csv',sep = '\t')

# get a list of all the WAV files in the directory
wav_files = [file for file in os.listdir(directory+ '/wavs') if file.endswith('.wav')]

for key,row in data.iterrows():
  if not row['wav'] in wav_files:
    data = data.drop(index=key,axis=0) 
print(data.__len__())


5235


In [ ]:
from datasets import load_dataset
from datasets import Dataset
dataset = Dataset.from_pandas(data)

In [ ]:
dataset

Dataset({
    features: ['wav', 'words', 'duration', '__index_level_0__'],
    num_rows: 5235
})

In [ ]:
dataset[0]

{'wav': 'clipping-306514b0-d498-41ff-b6ef-333cc36045d8.wav',
 'words': 'النار سمعت واحد قال لصاحبه رخص يكون عندنا شويه التراب',
 'duration': 5.934172335600908,
 '__index_level_0__': 0}

In [ ]:
from datasets import Audio

In [ ]:
audio_dataset = dataset.cast_column("wav", Audio())


In [ ]:
%cd /content/gdrive/MyDrive/'Colab Notebooks'/darija/wavs/

/content/gdrive/.shortcut-targets-by-id/1_vsMXjauFr0zXQ-Q9MMYFmt3nPqKPjQ0/darija/wavs


In [ ]:
audio_dataset

Dataset({
    features: ['wav', 'words', 'duration', '__index_level_0__'],
    num_rows: 5235
})

#3)Using Transfor;ers to generate the text in arabic langage

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 90.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.2 MB/s eta 0:00:00


In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Arabic", task="transcribe")

In [ ]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

do_lower_case = False
do_remove_punctuation = False

normalizer = BasicTextNormalizer()

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(model_name, language="Arabic", task="transcribe")

In [ ]:
def prepare_dataset(batch):
    # load and (possibly) resample wav data to 16kHz
    wav = batch["wav"]
    # compute log-Mel input features from input wav array 
    batch["input_features"] = processor.feature_extractor(wav["array"], sampling_rate=wav["sampling_rate"]).input_features[0]
    # compute input length of wav sample in seconds
    batch["input_length"] = len(wav["array"]) / wav["sampling_rate"]
    
    # optional pre-processing steps
    transcription = batch["words"]
    if do_lower_case:
        transcription = transcription.lower()
    if do_remove_punctuation:
        transcription = normalizer(transcription).strip()
    
    # encode target text to label ids
    batch["labels"] = processor.tokenizer(transcription).input_ids
    return batch

In [ ]:
audio_dataset

Dataset({
    features: ['wav', 'words', 'duration', '__index_level_0__'],
    num_rows: 5235
})

In [ ]:
audio_dataset[0]

{'wav': {'path': 'clipping-306514b0-d498-41ff-b6ef-333cc36045d8.wav',
  'array': array([-0.00048828, -0.00048828, -0.00048828, ..., -0.00231934,
         -0.00134277, -0.00180054]),
  'sampling_rate': 16000},
 'words': 'النار سمعت واحد قال لصاحبه رخص يكون عندنا شويه التراب',
 'duration': 5.934172335600908,
 '__index_level_0__': 0}

In [ ]:
min_dataset = audio_dataset.select(range(20))

In [ ]:
min_dataset = min_dataset.map(prepare_dataset, remove_columns=min_dataset.column_names, num_proc=12)


Map (num_proc=12):   0%|          | 0/20 [00:00<?, ? examples/s]

In [ ]:
max_input_length = 30.0

def is_audio_in_length_range(length):
    return length < max_input_length

In [ ]:
min_dataset = min_dataset.filter(
    is_audio_in_length_range,
    input_columns=["input_length"],)

Filter:   0%|          | 0/20 [00:00<?, ? examples/s]

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)


#4)Training the model :

In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.6 MB/s eta 0:00:00


In [ ]:
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 40.0 MB/s eta 0:00:00


In [ ]:
import evaluate
metric = evaluate.load("wer")


In [ ]:
# evaluate with the 'normalised' WER
do_normalize_eval = True

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    if do_normalize_eval:
        pred_str = [normalizer(pred) for pred in pred_str]
        label_str = [normalizer(label) for label in label_str]

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import WhisperForConditionalGeneration

#model = WhisperForConditionalGeneration.from_pretrained(model_name)

In [ ]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
model.config.use_cache = False

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="ADD",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=5,
    max_steps=50,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=6,
    eval_steps=6,
    logging_steps=5,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [ ]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid.
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as d

In [ ]:
data_eval = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/darija/texts/test.csv',sep = '\t')
# get a list of all the WAV files in the directory
wav_files = [file for file in os.listdir(directory) if file.endswith('.wav')]
for key,row in data_eval.iterrows():
  if not row['wav'] in wav_files:
    data_eval= data_eval.drop(index=key,axis=0) 
# dataset = load_dataset(, data_files="/content/gdrive/MyDrive/darija/texts/train.csv")
dataset = Dataset.from_pandas(data_eval)
eval_data = dataset.cast_column("wav", Audio())
mini_dataset = eval_data.select(range(20))
mini_dataset = mini_dataset.map(prepare_dataset, remove_columns=mini_dataset.column_names, num_proc=12)
mini_dataset = mini_dataset.filter(
    is_audio_in_length_range, 
    input_columns=["input_length"],)



In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=min_dataset,
    eval_dataset=mini_dataset.select(range(20)),
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

/content/gdrive/.shortcut-targets-by-id/1_vsMXjauFr0zXQ-Q9MMYFmt3nPqKPjQ0/darija/wavs/ADD is already a clone of https://huggingface.co/manal123/ADD. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
processor.save_pretrained(training_args.output_dir)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
6,2.639200,No log


KeyError: ignored

In [ ]:
model.push_to_hub('manal123/ADHD', use_auth_token=True)

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/967M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/manal123/ADHD/commit/b4834b06097d5a4f6dc5cb730ca126c3ca8f4f6f', commit_message='Upload WhisperForConditionalGeneration', commit_description='', oid='b4834b06097d5a4f6dc5cb730ca126c3ca8f4f6f', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
processor.push_to_hub('manal123/ADHD', use_auth_token=True)

CommitInfo(commit_url='https://huggingface.co/manal123/ADHD/commit/df67952e24962507053faa7fa4dac4740856d604', commit_message='Upload processor', commit_description='', oid='df67952e24962507053faa7fa4dac4740856d604', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
kwargs = {
    # "dataset_tags": "mozilla-foundation/common_voice_11_0",
    # "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "language": "ar",
    "model_name": "Whisper Small ar - adhd",  # a 'pretty' name for your model
    "finetuned_from": model_name,
    "tasks": "automatic-speech-recognition",
    # "tags": "whisper-event",
}

In [ ]:
processor.tokenizer

In [ ]:
model

In [ ]:
from transformers import pipeline
from huggingface_hub import model_info

MODEL_NAME = "arbml/whisper-medium-ar" #this always needs to stay in line 8 :D sorry for the hackiness
lang = "ar"

pipe = pipeline(
    tokenizer = processor.tokenizer,
    feature_extractor = processor.feature_extractor,
    task="automatic-speech-recognition",
    model=model,
    chunk_length_s=30,
    device="cuda:0",
)


#5)Building a user interface for our training model:

In [ ]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.9/287.9 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.

In [ ]:
text = pipe('/content/gdrive/MyDrive/Colab Notebooks/darija/wavs/1b43cac2-7d9c-463c-8e36-7f2fec9d2c6a.wav')["text"]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
text

'خيره اللي جزيكم بالخير و'

In [ ]:
DIR = '/content/gdrive/MyDrive/Colab Notebooks/darija/wavs/ADHD'

In [ ]:
model = WhisperForConditionalGeneration.from_pretrained(DIR)

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
import gradio as gr
from duckduckgo_search import ddg

def transcribe(audio):
    text = pipe(audio)["text"]

    results = ddg(text, region='wt-wt', safesearch='Off', time='y')
    titles = [t["title"] for t in results]
    return text + "\n" + "\n".join(titles)



gr.Interface(
    fn=transcribe,
    inputs=gr.Audio(source="microphone", type="filepath"), 
    outputs="text").launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

/usr/local/lib/python3.10/dist-packages/gradio/processing_utils.py:184: UserWarning: Trying to convert audio automatically from int32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/processing_utils.py:184: UserWarning: Trying to convert audio automatically from int32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers

Keyboard interruption in main thread... closing server.


#6)Linkng the output to a motor engine:

In [ ]:
pip install duckduckgo-search

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.27.1
    Uninstalling requests-2.27.1:
      Successfully uninstalled requests-2.27.1


In [ ]:
from duckduckgo_search import ddg

keywords = 'Bella Ciao'
results = ddg(keywords, region='wt-wt', safesearch='Off', time='y')
print(results)

[{'title': "Bella Ciao: A Brief History of Italy's Resistance Anthem - Wanted in Rome", 'href': 'https://www.wantedinrome.com/news/bella-ciao-a-brief-history-of-italys-resistance-anthem.html', 'body': "24 Apr, 2023 How a 19th-century protest folk song became the anthem of Italy's anti-fascist Resistance. Every year on 25 April, Bella Ciao is sung across Italy on the Festa della Liberazione which marks the country's liberation from German occupation and fascist rule at the end of world war two."}, {'title': 'Bella Ciao | The Song Of The Italian Resistance | Life In Italy', 'href': 'https://lifeinitaly.com/bella-ciao-from-the-resistance-to-money-heist/', 'body': 'Culture Music Bella Ciao, the meaning. Written by Gaia Zol on May 23, 2022 Bella Ciao, from the Resistance to Money Heist, back to the resistance. Find out more about this iconic song. Where does it come from? And what does it mean? The Netflix show brought Bella Ciao to the international song, but protesters all over the world 